---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# Data loading and processing
Cols = ['ticket_id', 'fine_amount','admin_fee','state_fee','late_fee', 'discount_amount']
X_Val = pd.read_csv('test.csv', usecols=Cols)
X_Val.set_index(['ticket_id'], inplace=True)

Cols = ['ticket_id', 'fine_amount','admin_fee','state_fee','late_fee', 'discount_amount' ,'compliance']
TrainD = pd.read_csv('train.csv', usecols=Cols)
TrainD.dropna(inplace=True)

Comp = TrainD['compliance'].copy()

TrainD.drop(['compliance'], axis=1, inplace=True)
TrainD.set_index(['ticket_id'], inplace=True)


In [3]:
# Random forest model hyperparameter tuning by gridsearch and CV
# Model tuned using ROC AUC score

X_train, X_test, y_train, y_test = train_test_split(TrainD, Comp, random_state=0)
    
model = RandomForestClassifier() #Define RFC model
grid_values = {'n_estimators': [1, 5, 10, 100,200,300,500], 'max_features': ['sqrt',1,2,3]} #Define values for grid search

grid_clf_auc = GridSearchCV(model, param_grid = grid_values, scoring = 'roc_auc', verbose=2) #Define the grid search object
grid_clf_auc.fit(X_train, y_train) #Fit the model

y_proba_tr = grid_clf_auc.predict_proba(X_train) #Predict probabilities of train set
Faults1 = np.array([s[1] for s in  y_proba_tr]) #Get the probabilities of ticket paid

y_proba_te = grid_clf_auc.predict_proba(X_test) #Predict probabilities of test set
Faults = np.array([s[1] for s in  y_proba_te]) #Get the probabilities of ticket paid

print('Train set AUC: ', roc_auc_score(y_train, Faults1))
print('Test set AUC: ', roc_auc_score(y_test, Faults))
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

ures=sqrt, n_estimators=1 ...............................
[CV] ................ max_features=sqrt, n_estimators=1, total=   0.0s
[CV] max_features=sqrt, n_estimators=1 ...............................
[CV] ................ max_features=sqrt, n_estimators=1, total=   0.0s
[CV] max_features=sqrt, n_estimators=1 ...............................
[CV] ................ max_features=sqrt, n_estimators=1, total=   0.0s
[CV] max_features=sqrt, n_estimators=5 ...............................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] ................ max_features=sqrt, n_estimators=5, total=   0.1s
[CV] max_features=sqrt, n_estimators=5 ...............................
[CV] ................ max_features=sqrt, n_estimators=5, total=   0.1s
[CV] max_features=sqrt, n_estimators=5 ...............................
[CV] ................ max_features=sqrt, n_estimators=5, total=   0.1s


In [6]:
def blight_model():
    # The optimal model is defined and trained with the whole train set
    Model = RandomForestClassifier(n_estimators=100, max_features=1).fit(TrainD, Comp) 
    
    proba = Model.predict_proba(X_Val) #Calculate Test probabilites
    Pays = np.array([s[1] for s in  proba]) #Get ticket paid probabilites
    probs = pd.Series(Pays, index=X_Val.index) #Reorder data for submission
    
    return probs

In [7]:
blight_model()

ticket_id
284932    0.060604
285362    0.023615
285361    0.069553
285338    0.060604
285346    0.069553
            ...   
376496    0.023615
376497    0.023615
376499    0.069553
376500    0.069553
369851    0.261616
Length: 61001, dtype: float64